In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('MNT_TEST_zero_shot_round2_withcommas.csv')
# 'withcommas' means that the csv is comma-separated and not e.g. tab-separated
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>..."
2,2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...
3,3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...
4,4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...


In [3]:
def split_negation(sentence):
    sentence = sentence.replace("couldn't <O>\n", "could <O>\nn't <O>\n")
    sentence = sentence.replace("shouldn't <O>\n", "should <O>\nn't <O>\n")
    sentence = sentence.replace("wouldn't <O>\n", "would <O>\nn't <O>\n")
    sentence = sentence.replace("don't <O>\n", "do <O>\nn't<O>\n")
    sentence = sentence.replace("doesn't <O>\n", "does <O>\nn't <O>\n")
    sentence = sentence.replace("isn't <O>\n", "is <O>\nn't <O>\n")
    sentence = sentence.replace("aren't <O>\n", "are <O>\nn't <O>\n")
    sentence = sentence.replace("didn't <O>\n", "did <O>\nn't <O>\n")
    sentence = sentence.replace("wasn't <O>\n", "was <O>\nn't <O>\n")
    sentence = sentence.replace("weren't <O>\n", "were <O>\nn't <O>\n")
    sentence = sentence.replace("hasn't <O>\n", "has <O>\nn't <O>\n")
    sentence = sentence.replace("haven't <O>\n", "have <O>\nn't <O>\n")
    sentence = sentence.replace("hadn't <O>\n", "had <O>\nn't <O>\n")
    if "\n's <O>\n" in sentence:
        sentence = sentence.replace("\n's <O>\n", "\n's <O>\n")
    else:
        sentence = sentence.replace("'s <O>\n", " <O>\n's <O>\n")
    return sentence

In [4]:
df['LLM_response_split_negation'] = df['LLM_response'].apply(split_negation)
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>..."
2,2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...
3,3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...
4,4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...


In [5]:
print(df['LLM_response_split_negation'][5])

JSA <O>
Research <O>
defence <O>
analysist <O>
Paul <O>
Nesbit <O>
said <EVENT>
that <O>
Maintenance <O>
Repair <O>
and <O>
Overhaul <O>
(MRO) <O>
operations <O>
across <O>
the <O>
aircraft <O>
's <O>
fifty-year <O>
service <O>
life <O>
could <O>
push <O>
the <O>
value <O>
of <O>
the <O>
contract <O>
as <O>
high <O>
as <O>
US$ <O>
100 <O>
billion <O>
Indonesia <O>
's <O>
transport <O>
minister <O>
tells <EVENT>
airlines <O>
not <O>
to <O>
buy <O>
European <O>
aircraft <O>
due <O>
to <O>
EU <O>
ban <EVENT>
<>. <O>
<>, <O>


In [6]:
def token_tag(annotated_sentence):
    lines = annotated_sentence.split('\n')
    list_of_token_tag = []
    for line in lines:
        line = line.split()
        if len(line) == 2:
            if 'EVENT' in line[1] or 'O' in line[1]:
                list_of_token_tag.append(line)
                list_of_token_tag
    tuples = []
    for pair in list_of_token_tag:
        tuples.append(tuple(pair))
    return tuple(tuples) # I am using tuples for further processing, see zip()

In [7]:
df['list_of_token_tag_pairs'] = df['LLM_response_split_negation'].apply(token_tag)

In [8]:
(df['list_of_token_tag_pairs'] == ()).sum()

110

In [9]:
df_errors = df.loc[df['list_of_token_tag_pairs'] == (), :].reset_index()
df_errors.head()

,index,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs
0,9,9,140538816301616,The EU made the decision to ban the Indonesian...,Given the sentence between the brackets <The E...,The\n<EVENT>\nEU\n<EVENT>\nmade\n<EVENT>\nthe\...,The\n<EVENT>\nEU\n<EVENT>\nmade\n<EVENT>\nthe\...,()
1,10,10,140538816336432,`` We do not need to buy aircraft from Europe ...,Given the sentence between the brackets <`` We...,We\n<O>\ndo\n<O>\nnot\n<O>\nneed\n<O>\nto\n<O>...,We\n<O>\ndo\n<O>\nnot\n<O>\nneed\n<O>\nto\n<O>...,()
2,11,11,140538816684720,"March 27, 2008",Given the sentence between the brackets <March...,March\n<EVENT>\n27\n<EVENT>\n2008\n<EVENT>\n> ...,March\n<EVENT>\n27\n<EVENT>\n2008\n<EVENT>\n> ...,()
3,36,36,140538816684976,"May 24, 2008",Given the sentence between the brackets <May 2...,"May <EVENT> 24 <O> 2008 <O>,","May <EVENT> 24 <O> 2008 <O>,",()
4,49,49,140538818115856,The Dreamliner will be the first commercial je...,Given the sentence between the brackets <The D...,The\n<EVENT>\nDreamliner\n<EVENT>\nwill\n<O>\n...,The\n<EVENT>\nDreamliner\n<EVENT>\nwill\n<O>\n...,()


In [10]:
def error_type(annotated_sentence):
    lines = annotated_sentence.split('\n')
    for line in lines:
        line = line.split()
        if len(line) < 2:
            return 'each token and each tag is in a separate line'
        else:
            return 'the whole sentence is in one continuous line'
            # it might be better to say that there are multiple tokens in one line
    return

In [11]:
df_errors['error_type'] = df_errors['LLM_response_split_negation'].apply(error_type)
df_errors.head()

,index,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,error_type
0,9,9,140538816301616,The EU made the decision to ban the Indonesian...,Given the sentence between the brackets <The E...,The\n<EVENT>\nEU\n<EVENT>\nmade\n<EVENT>\nthe\...,The\n<EVENT>\nEU\n<EVENT>\nmade\n<EVENT>\nthe\...,(),each token and each tag is in a separate line
1,10,10,140538816336432,`` We do not need to buy aircraft from Europe ...,Given the sentence between the brackets <`` We...,We\n<O>\ndo\n<O>\nnot\n<O>\nneed\n<O>\nto\n<O>...,We\n<O>\ndo\n<O>\nnot\n<O>\nneed\n<O>\nto\n<O>...,(),each token and each tag is in a separate line
2,11,11,140538816684720,"March 27, 2008",Given the sentence between the brackets <March...,March\n<EVENT>\n27\n<EVENT>\n2008\n<EVENT>\n> ...,March\n<EVENT>\n27\n<EVENT>\n2008\n<EVENT>\n> ...,(),each token and each tag is in a separate line
3,36,36,140538816684976,"May 24, 2008",Given the sentence between the brackets <May 2...,"May <EVENT> 24 <O> 2008 <O>,","May <EVENT> 24 <O> 2008 <O>,",(),the whole sentence is in one continuous line
4,49,49,140538818115856,The Dreamliner will be the first commercial je...,Given the sentence between the brackets <The D...,The\n<EVENT>\nDreamliner\n<EVENT>\nwill\n<O>\n...,The\n<EVENT>\nDreamliner\n<EVENT>\nwill\n<O>\n...,(),each token and each tag is in a separate line


In [12]:
df_errors['error_type'].value_counts(normalize = True)

each token and each tag is in a separate line    0.727273
the whole sentence is in one continuous line     0.272727
Name: error_type, dtype: float64

In [14]:
with open('test_tags_adjusted_MNT.txt', encoding = 'utf8') as infile:
    gold_MNT_TEST = infile.read()
print(gold_MNT_TEST[:100])

Northrop	<O>
Grumman	<O>
and	<O>
Airbus	<O>
parent	<O>
EADS	<O>
defeat	<EVENT>
Boeing	<O>
for	<O>
$	


In [15]:
gold_MNT_TEST = gold_MNT_TEST.split('\n\n')

In [16]:
df2 = pd.DataFrame({'gold_MNT_TEST': gold_MNT_TEST})
df2

,gold_MNT_TEST
0,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...
1,"March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>"
2,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...
3,It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...
4,That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...
...,...
1110,"September\t<O>\n18\t<O>\n,\t<O>\n2008\t<O>"
1111,Stock\t<O>\nindexes\t<O>\naround\t<O>\nthe\t<O...
1112,"At\t<O>\n0800\t<O>\nUTC\t<O>\n,\t<O>\nthe\t<O>..."
1113,The\t<O>\ncredit\t<O>\narrangements\t<EVENT>\n...


In [17]:
df['gold_MNT_TEST'] = df2['gold_MNT_TEST']
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_MNT_TEST
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,"((North, <O>), (Grumman, <O>), (and, <O>), (Ai...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>"
2,2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,"((and, <O>), (Airbus, <O>), (parent, <O>), (co...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...
3,3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,"((It, <EVENT>), (is, <O>), (the, <O>), (bigges...",It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...
4,4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,"((That, EVENT), (was, EVENT), (contract, EVENT...",That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...


In [18]:
df['gold_list_of_token_tag_pairs'] = df['gold_MNT_TEST'].apply(token_tag)
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_MNT_TEST,gold_list_of_token_tag_pairs
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,"((North, <O>), (Grumman, <O>), (and, <O>), (Ai...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ..."
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","((March, <O>), (01, <O>), (,, <O>), (2008, <O>))"
2,2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,"((and, <O>), (Airbus, <O>), (parent, <O>), (co...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ..."
3,3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,"((It, <EVENT>), (is, <O>), (the, <O>), (bigges...",It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...,"((It, <O>), (is, <EVENT>), (the, <O>), (bigges..."
4,4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,"((That, EVENT), (was, EVENT), (contract, EVENT...",That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...,"((That, <O>), (contract, <O>), (was, <O>), (fo..."


In [19]:
# remove commas and fullstops from tuple lists to make the postprocessing feasible
def remove_commas_fullstops(tuple_of_tuples):
    list_of_tuples = list(tuple_of_tuples)
    if len(tuple_of_tuples) !=0:
        for item in list_of_tuples:
            list_without_commas_and_fullstops = [item for item in list_of_tuples if item[0] != ',' and item[0] != '.']
        return tuple(list_without_commas_and_fullstops)
    else:
        return tuple_of_tuples
    return    

In [20]:
df['list_of_token_tag_pairs_without_commas_fullstops'] = df['list_of_token_tag_pairs'].apply(remove_commas_fullstops)
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_MNT_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,"((North, <O>), (Grumman, <O>), (and, <O>), (Ai...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((North, <O>), (Grumman, <O>), (and, <O>), (Ai..."
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","((March, <O>), (01, <O>), (,, <O>), (2008, <O>))","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN..."
2,2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,"((and, <O>), (Airbus, <O>), (parent, <O>), (co...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((and, <O>), (Airbus, <O>), (parent, <O>), (co..."
3,3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,"((It, <EVENT>), (is, <O>), (the, <O>), (bigges...",It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...,"((It, <O>), (is, <EVENT>), (the, <O>), (bigges...","((It, <EVENT>), (is, <O>), (the, <O>), (bigges..."
4,4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,"((That, EVENT), (was, EVENT), (contract, EVENT...",That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...,"((That, <O>), (contract, <O>), (was, <O>), (fo...","((That, EVENT), (was, EVENT), (contract, EVENT..."


In [21]:
df['gold_list_of_token_tag_pairs_without_commas_fullstops'] = df['gold_list_of_token_tag_pairs'].apply(remove_commas_fullstops)
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_MNT_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,"((North, <O>), (Grumman, <O>), (and, <O>), (Ai...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((North, <O>), (Grumman, <O>), (and, <O>), (Ai...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ..."
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","((March, <O>), (01, <O>), (,, <O>), (2008, <O>))","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","((March, <O>), (01, <O>), (2008, <O>))"
2,2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,"((and, <O>), (Airbus, <O>), (parent, <O>), (co...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((and, <O>), (Airbus, <O>), (parent, <O>), (co...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ..."
3,3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,"((It, <EVENT>), (is, <O>), (the, <O>), (bigges...",It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...,"((It, <O>), (is, <EVENT>), (the, <O>), (bigges...","((It, <EVENT>), (is, <O>), (the, <O>), (bigges...","((It, <O>), (is, <EVENT>), (the, <O>), (bigges..."
4,4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,"((That, EVENT), (was, EVENT), (contract, EVENT...",That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...,"((That, <O>), (contract, <O>), (was, <O>), (fo...","((That, EVENT), (was, EVENT), (contract, EVENT...","((That, <O>), (contract, <O>), (was, <O>), (fo..."


In [22]:
equal_length = []
for idx, row in df.iloc[0:].iterrows():
    a=row['list_of_token_tag_pairs']
    b=row['gold_list_of_token_tag_pairs']
    x = len(a) == len(b)
    equal_length.append(x)   

In [23]:
print(equal_length[:50])

[False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, True, False, True, True, False, False, False, True, False, False, False]


In [24]:
df['equal_length']= pd.DataFrame({
    "equal_length": equal_length
})
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_MNT_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops,equal_length
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,"((North, <O>), (Grumman, <O>), (and, <O>), (Ai...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((North, <O>), (Grumman, <O>), (and, <O>), (Ai...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...",False
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","((March, <O>), (01, <O>), (,, <O>), (2008, <O>))","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","((March, <O>), (01, <O>), (2008, <O>))",False
2,2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,"((and, <O>), (Airbus, <O>), (parent, <O>), (co...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((and, <O>), (Airbus, <O>), (parent, <O>), (co...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...",False
3,3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,"((It, <EVENT>), (is, <O>), (the, <O>), (bigges...",It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...,"((It, <O>), (is, <EVENT>), (the, <O>), (bigges...","((It, <EVENT>), (is, <O>), (the, <O>), (bigges...","((It, <O>), (is, <EVENT>), (the, <O>), (bigges...",False
4,4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,"((That, EVENT), (was, EVENT), (contract, EVENT...",That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...,"((That, <O>), (contract, <O>), (was, <O>), (fo...","((That, EVENT), (was, EVENT), (contract, EVENT...","((That, <O>), (contract, <O>), (was, <O>), (fo...",False


In [25]:
df['equal_length'].value_counts()

False    899
True     216
Name: equal_length, dtype: int64

In [26]:
#shorter than gold
shorter_than_gold = []
for idx, row in df.iloc[0:].iterrows():
    a=row['list_of_token_tag_pairs']
    b=row['gold_list_of_token_tag_pairs']
    y = len(a) < len(b)
    shorter_than_gold.append(y)

df['shorter_than_gold']= pd.DataFrame({
    "shorter_than_gold": shorter_than_gold
})
df.head()

df['shorter_than_gold'].value_counts()

True     844
False    271
Name: shorter_than_gold, dtype: int64

In [27]:
#longer than gold
longer_than_gold = []
for idx, row in df.iloc[0:].iterrows():
    a=row['list_of_token_tag_pairs']
    b=row['gold_list_of_token_tag_pairs']
    z = len(a) > len(b)
    longer_than_gold.append(z)

df['longer_than_gold']= pd.DataFrame({
    "longer_than_gold": longer_than_gold
})
df.head()

df['longer_than_gold'].value_counts()

False    1060
True       55
Name: longer_than_gold, dtype: int64

In [28]:
#equal length without commas and fullstops
equal_length_without_commas_fullstops= []
for idx, row in df.iloc[0:].iterrows():
    a=row['list_of_token_tag_pairs_without_commas_fullstops']
    b=row['gold_list_of_token_tag_pairs_without_commas_fullstops']
    x = len(a) == len(b)
    equal_length_without_commas_fullstops.append(x)

In [29]:
df['equal_length_without_commas_fullstops']= pd.DataFrame({
    "equal_length_without_commas_fullstops": equal_length_without_commas_fullstops
})
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_MNT_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops,equal_length,shorter_than_gold,longer_than_gold,equal_length_without_commas_fullstops
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,"((North, <O>), (Grumman, <O>), (and, <O>), (Ai...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((North, <O>), (Grumman, <O>), (and, <O>), (Ai...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...",False,False,True,False
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","((March, <O>), (01, <O>), (,, <O>), (2008, <O>))","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","((March, <O>), (01, <O>), (2008, <O>))",False,False,True,False
2,2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,"((and, <O>), (Airbus, <O>), (parent, <O>), (co...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((and, <O>), (Airbus, <O>), (parent, <O>), (co...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...",False,True,False,False
3,3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,"((It, <EVENT>), (is, <O>), (the, <O>), (bigges...",It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...,"((It, <O>), (is, <EVENT>), (the, <O>), (bigges...","((It, <EVENT>), (is, <O>), (the, <O>), (bigges...","((It, <O>), (is, <EVENT>), (the, <O>), (bigges...",False,True,False,False
4,4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,"((That, EVENT), (was, EVENT), (contract, EVENT...",That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...,"((That, <O>), (contract, <O>), (was, <O>), (fo...","((That, EVENT), (was, EVENT), (contract, EVENT...","((That, <O>), (contract, <O>), (was, <O>), (fo...",False,True,False,False


In [30]:
df['equal_length_without_commas_fullstops'].value_counts()

False    724
True     391
Name: equal_length_without_commas_fullstops, dtype: int64

In [31]:
complete_dataset_for_eval= []
for idx, row in df.iloc[0:].iterrows():
    a=row['equal_length_without_commas_fullstops']
    b=row['LLM_response_split_negation']
    c=row['gold_MNT_TEST']
    if a == True:
        complete_dataset_for_eval.append(b)
    else:
        complete_dataset_for_eval.append(c)

df['complete_dataset_for_eval'] = complete_dataset_for_eval

In [32]:
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_MNT_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops,equal_length,shorter_than_gold,longer_than_gold,equal_length_without_commas_fullstops,complete_dataset_for_eval
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,"((North, <O>), (Grumman, <O>), (and, <O>), (Ai...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((North, <O>), (Grumman, <O>), (and, <O>), (Ai...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...",False,False,True,False,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","((March, <O>), (01, <O>), (,, <O>), (2008, <O>))","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","((March, <O>), (01, <O>), (2008, <O>))",False,False,True,False,"March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>"
2,2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,"((and, <O>), (Airbus, <O>), (parent, <O>), (co...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((and, <O>), (Airbus, <O>), (parent, <O>), (co...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...",False,True,False,False,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...
3,3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,"((It, <EVENT>), (is, <O>), (the, <O>), (bigges...",It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...,"((It, <O>), (is, <EVENT>), (the, <O>), (bigges...","((It, <EVENT>), (is, <O>), (the, <O>), (bigges...","((It, <O>), (is, <EVENT>), (the, <O>), (bigges...",False,True,False,False,It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...
4,4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,"((That, EVENT), (was, EVENT), (contract, EVENT...",That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...,"((That, <O>), (contract, <O>), (was, <O>), (fo...","((That, EVENT), (was, EVENT), (contract, EVENT...","((That, <O>), (contract, <O>), (was, <O>), (fo...",False,True,False,False,That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...


In [33]:
# where equal_length_without_commas_fullstops is False, change EVENT to O
complete_dataset_for_eval_adjusted= []
for idx, row in df.iloc[0:].iterrows():
    a=row['equal_length_without_commas_fullstops']
    b=row['complete_dataset_for_eval']
    if a == False:
        o_tags = b.replace("<EVENT>", "<O>")
        complete_dataset_for_eval_adjusted.append(o_tags)
    else:
        complete_dataset_for_eval_adjusted.append(b)

df['complete_dataset_for_eval_adjusted'] = complete_dataset_for_eval_adjusted
df.head(2)

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_MNT_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops,equal_length,shorter_than_gold,longer_than_gold,equal_length_without_commas_fullstops,complete_dataset_for_eval,complete_dataset_for_eval_adjusted
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,"((North, <O>), (Grumman, <O>), (and, <O>), (Ai...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((North, <O>), (Grumman, <O>), (and, <O>), (Ai...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...",False,False,True,False,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","((March, <O>), (01, <O>), (,, <O>), (2008, <O>))","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","((March, <O>), (01, <O>), (2008, <O>))",False,False,True,False,"March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>"


In [34]:
df.head(10)

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_MNT_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops,equal_length,shorter_than_gold,longer_than_gold,equal_length_without_commas_fullstops,complete_dataset_for_eval,complete_dataset_for_eval_adjusted
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,"((North, <O>), (Grumman, <O>), (and, <O>), (Ai...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((North, <O>), (Grumman, <O>), (and, <O>), (Ai...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...",False,False,True,False,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","((March, <O>), (01, <O>), (,, <O>), (2008, <O>))","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","((March, <O>), (01, <O>), (2008, <O>))",False,False,True,False,"March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>"
2,2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,"((and, <O>), (Airbus, <O>), (parent, <O>), (co...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((and, <O>), (Airbus, <O>), (parent, <O>), (co...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...",False,True,False,False,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...
3,3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,"((It, <EVENT>), (is, <O>), (the, <O>), (bigges...",It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...,"((It, <O>), (is, <EVENT>), (the, <O>), (bigges...","((It, <EVENT>), (is, <O>), (the, <O>), (bigges...","((It, <O>), (is, <EVENT>), (the, <O>), (bigges...",False,True,False,False,It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...,It\t<O>\nis\t<O>\nthe\t<O>\nbiggest\t<O>\ncont...
4,4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,"((That, EVENT), (was, EVENT), (contract, EVENT...",That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...,"((That, <O>), (contract, <O>), (was, <O>), (fo...","((That, EVENT), (was, EVENT), (contract, EVENT...","((That, <O>), (contract, <O>), (was, <O>), (fo...",False,True,False,False,That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...,That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<O...
5,5,140538820797424,JSA Research defence analysist Paul Nesbit sai...,Given the sentence between the brackets <JSA R...,JSA <O>\nResearch <O>\ndefence <O>\nanalysist ...,JSA <O>\nResearch <O>\ndefence <O>\nanalysist ...,"((JSA, <O>), (Research, <O>), (defence, <O>), ...",JSA\t<O>\nResearch\t<O>\ndefence\t<O>\nanalysi...,"((JSA, <O>), (Research, <O>), (defence, <O>), ...","((JSA, <O>), (Research, <O>), (defence, <O>), ...","((JSA, <O>), (Research, <O>), (defence, <O>), ...",False,True,False,True,JSA <O>\nResearch <O>\ndefence <O>\nanalysist ...,JSA <O>\nResearch <O>\ndefence <O>\nanalysist ...
6,6,140541903195792,"December 20, 2007",Given the sentence between the bra

In [35]:
print(df['complete_dataset_for_eval_adjusted'][9])

The	<O>
EU	<O>
made	<O>
the	<O>
decision	<O>
to	<O>
ban	<O>
the	<O>
Indonesian	<O>
air	<O>
traffic	<O>
due	<O>
to	<O>
a	<O>
string	<O>
of	<O>
accidents	<O>
in	<O>
the	<O>
country	<O>
,	<O>
including	<O>
two	<O>
major	<O>
disasters	<O>
this	<O>
year	<O>
-	<O>
the	<O>
losses	<O>
of	<O>
Adam	<O>
Air	<O>
Flight	<O>
574	<O>
and	<O>
Garuda	<O>
Indonesia	<O>
Flight	<O>
200	<O>
,	<O>
which	<O>
killed	<O>
102	<O>
and	<O>
21	<O>
people	<O>
respectively	<O>
.	<O>


In [36]:
print(df['complete_dataset_for_eval'][9])

The	<O>
EU	<O>
made	<EVENT>
the	<O>
decision	<EVENT>
to	<O>
ban	<EVENT>
the	<O>
Indonesian	<O>
air	<O>
traffic	<O>
due	<O>
to	<O>
a	<O>
string	<O>
of	<O>
accidents	<EVENT>
in	<O>
the	<O>
country	<O>
,	<O>
including	<O>
two	<O>
major	<O>
disasters	<EVENT>
this	<O>
year	<O>
-	<O>
the	<O>
losses	<EVENT>
of	<O>
Adam	<O>
Air	<O>
Flight	<EVENT>
574	<O>
and	<O>
Garuda	<O>
Indonesia	<O>
Flight	<EVENT>
200	<O>
,	<O>
which	<EVENT>
killed	<EVENT>
102	<O>
and	<O>
21	<O>
people	<O>
respectively	<O>
.	<O>


In [37]:
df.shape

(1115, 17)

In [38]:
# list of token tag pairs
df['for_eval_token_tag_pairs'] = df['complete_dataset_for_eval_adjusted'].apply(token_tag)

In [39]:
# remove commas and fullstops
df['for_eval_token_tag_pairs_without_commas_fullstops'] = df['for_eval_token_tag_pairs'].apply(remove_commas_fullstops)
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_MNT_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops,equal_length,shorter_than_gold,longer_than_gold,equal_length_without_commas_fullstops,complete_dataset_for_eval,complete_dataset_for_eval_adjusted,for_eval_token_tag_pairs,for_eval_token_tag_pairs_without_commas_fullstops
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,"((North, <O>), (Grumman, <O>), (and, <O>), (Ai...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((North, <O>), (Grumman, <O>), (and, <O>), (Ai...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...",False,False,True,False,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ..."
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","((March, <O>), (01, <O>), (,, <O>), (2008, <O>))","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","((March, <O>), (01, <O>), (2008, <O>))",False,False,True,False,"March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","((March, <O>), (01, <O>), (,, <O>), (2008, <O>))","((March, <O>), (01, <O>), (2008, <O>))"
2,2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,"((and, <O>), (Airbus, <O>), (parent, <O>), (co...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((and, <O>), (Airbus, <O>), (parent, <O>), (co...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...",False,True,False,False,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ..."
3,3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,"((It, <EVENT>), (is, <O>), (the, <O>), (bigges...",It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...,"((It, <O>), (is, <EVENT>), (the, <O>), (bigges...","((It, <EVENT>), (is, <O>), (the, <O>), (bigges...","((It, <O>), (is, <EVENT>), (the, <O>), (bigges...",False,True,False,False,It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...,It\t<O>\nis\t<O>\nthe\t<O>\nbiggest\t<O>\ncont...,"((It, <O>), (is, <O>), (the, <O>), (biggest, <...","((It, <O>), (is, <O>), (the, <O>), (biggest, <..."
4,4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,"((That, EVENT), (was, EVENT), (contract, EVENT...",That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...,"((That, <O>), (contract, <O>), (was, <O>), (fo...","((That, EVENT), (was, EVENT), (contract, EVENT...","((That, <O>), (contract, <O>), (was, <O>), (fo...",False,True,False,False,That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...,That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<O...,"((That, <O>), (contract, <O>), (was, <O>), (fo...","((That, <O>), (contract, <O>), (was, <O>), (fo..."


In [40]:
# equal length without commas and fullstops
equal_length_without_commas_fullstops= []
for idx, row in df.iloc[0:].iterrows():
    a=row['for_eval_token_tag_pairs_without_commas_fullstops']
    b=row['gold_list_of_token_tag_pairs_without_commas_fullstops']
    x = len(a) == len(b)
    equal_length_without_commas_fullstops.append(x)

In [41]:
# check equal length one more time
equal_length_without_commas_fullstops.count(True)

1115

In [42]:
equal_length_without_commas_fullstops.count(False)

0

In [43]:
zippedlines = []
for idx, row in df.iloc[0:].iterrows():    
    a=row['for_eval_token_tag_pairs_without_commas_fullstops']
    b=row['gold_list_of_token_tag_pairs_without_commas_fullstops']
    a= list(a)
    b= list(b)
    zippedlines.append(list(zip(b, a)))

In [44]:
df['zippedlines']= pd.DataFrame({
    "zippedlines": zippedlines
})
df

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_MNT_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops,equal_length,shorter_than_gold,longer_than_gold,equal_length_without_commas_fullstops,complete_dataset_for_eval,complete_dataset_for_eval_adjusted,for_eval_token_tag_pairs,for_eval_token_tag_pairs_without_commas_fullstops,zippedlines
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...,"((North, <O>), (Grumman, <O>), (and, <O>), (Ai...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((North, <O>), (Grumman, <O>), (and, <O>), (Ai...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...",False,False,True,False,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","[((Northrop, <O>), (Northrop, <O>)), ((Grumman..."
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>...","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","((March, <O>), (01, <O>), (,, <O>), (2008, <O>))","((March, <EVENT>), (01, <EVENT>), (2008, <EVEN...","((March, <O>), (01, <O>), (2008, <O>))",False,False,True,False,"March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","March\t<O>\n01\t<O>\n,\t<O>\n2008\t<O>","((March, <O>), (01, <O>), (,, <O>), (2008, <O>))","((March, <O>), (01, <O>), (2008, <O>))","[((March, <O>), (March, <O>)), ((01, <O>), (01..."
2,2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...,"((and, <O>), (Airbus, <O>), (parent, <O>), (co...",Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((and, <O>), (Airbus, <O>), (parent, <O>), (co...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...",False,True,False,False,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,Northrop\t<O>\nGrumman\t<O>\nand\t<O>\nAirbus\...,"((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","((Northrop, <O>), (Grumman, <O>), (and, <O>), ...","[((Northrop, <O>), (Northrop, <O>)), ((Grumman..."
3,3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...,"((It, <EVENT>), (is, <O>), (the, <O>), (bigges...",It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...,"((It, <O>), (is, <EVENT>), (the, <O>), (bigges...","((It, <EVENT>), (is, <O>), (the, <O>), (bigges...","((It, <O>), (is, <EVENT>), (the, <O>), (bigges...",False,True,False,False,It\t<O>\nis\t<EVENT>\nthe\t<O>\nbiggest\t<O>\n...,It\t<O>\nis\t<O>\nthe\t<O>\nbiggest\t<O>\ncont...,"((It, <O>), (is, <O>), (the, <O>), (biggest, <...","((It, <O>), (is, <O>), (the, <O>), (biggest, <...","[((It, <O>), (It, <O>)), ((is, <EVENT>), (is, ..."
4,4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...,"((That, EVENT), (was, EVENT), (contract, EVENT...",That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...,"((That, <O>), (contract, <O>), (was, <O>), (fo...","((That, EVENT), (was, EVENT), (contract, EVENT...","((That, <O>), (contract, <O>), (was, <O>), (fo...",False,True,False,False,That\t<O>\ncontract\t<O>\nwas\t<O>\nfought\t<E...,That\t<O>\ncontract\t<O>\

In [45]:
df['zippedlines'][0]

[(('Northrop', '<O>'), ('Northrop', '<O>')),
 (('Grumman', '<O>'), ('Grumman', '<O>')),
 (('and', '<O>'), ('and', '<O>')),
 (('Airbus', '<O>'), ('Airbus', '<O>')),
 (('parent', '<O>'), ('parent', '<O>')),
 (('EADS', '<O>'), ('EADS', '<O>')),
 (('defeat', '<EVENT>'), ('defeat', '<O>')),
 (('Boeing', '<O>'), ('Boeing', '<O>')),
 (('for', '<O>'), ('for', '<O>')),
 (('$', '<O>'), ('$', '<O>')),
 (('40', '<O>'), ('40', '<O>')),
 (('billion', '<O>'), ('billion', '<O>')),
 (('US', '<O>'), ('US', '<O>')),
 (('airtanker', '<O>'), ('airtanker', '<O>')),
 (('contract', '<O>'), ('contract', '<O>'))]

In [46]:
def zips(tuples):
    lines = []
    for item in tuples:
        lines.append(f'{id(item)}\t{item[0][0]}\t{item[0][1]}\t{item[1][0]}\t{item[1][1]}\n')
    return ''.join(lines)

In [47]:
for idx, row in df.iloc[0:].iterrows():    
    a=row['zippedlines']
    a= zips(a)
    with open('MNT_zero_shot_gold_predicted_1706.txt', 'a', encoding='utf8') as outfile:
        outfile.write(f'{a}\n')    

In [48]:
with open('MNT_zero_shot_gold_predicted_1706.txt', encoding='utf8') as infile:
    text = infile.read()

In [49]:
print(text[:2000])

1897576699584	Northrop	<O>	Northrop	<O>
1897576706496	Grumman	<O>	Grumman	<O>
1897576384128	and	<O>	and	<O>
1897571141824	Airbus	<O>	Airbus	<O>
1897576704448	parent	<O>	parent	<O>
1897576704128	EADS	<O>	EADS	<O>
1897576703744	defeat	<EVENT>	defeat	<O>
1897576703616	Boeing	<O>	Boeing	<O>
1897576702912	for	<O>	for	<O>
1897576699648	$	<O>	$	<O>
1897576306240	40	<O>	40	<O>
1897576703296	billion	<O>	billion	<O>
1897576296512	US	<O>	US	<O>
1897576698624	airtanker	<O>	airtanker	<O>
1897576701760	contract	<O>	contract	<O>

1897571439936	March	<O>	March	<O>
1897576699840	01	<O>	01	<O>
1897571438912	2008	<O>	2008	<O>

1897576701312	Northrop	<O>	Northrop	<O>
1897565176192	Grumman	<O>	Grumman	<O>
1897576700416	and	<O>	and	<O>
1897576700672	Airbus	<O>	Airbus	<O>
1897576700352	parent	<O>	parent	<O>
1897576323840	company	<O>	company	<O>
1897576699328	European	<O>	European	<O>
1897576700736	Aeronautic	<O>	Aeronautic	<O>
1897576695040	Defence	<O>	Defence	<O>
1897576699264	and	<O>	and	<O>
1897576703936	

In [50]:
text = text.split('\n')

In [51]:
lines = []
for item in text:
    line = item.split()
    lines.append(line)
    
print(lines[:20])

[['1897576699584', 'Northrop', '<O>', 'Northrop', '<O>'], ['1897576706496', 'Grumman', '<O>', 'Grumman', '<O>'], ['1897576384128', 'and', '<O>', 'and', '<O>'], ['1897571141824', 'Airbus', '<O>', 'Airbus', '<O>'], ['1897576704448', 'parent', '<O>', 'parent', '<O>'], ['1897576704128', 'EADS', '<O>', 'EADS', '<O>'], ['1897576703744', 'defeat', '<EVENT>', 'defeat', '<O>'], ['1897576703616', 'Boeing', '<O>', 'Boeing', '<O>'], ['1897576702912', 'for', '<O>', 'for', '<O>'], ['1897576699648', '$', '<O>', '$', '<O>'], ['1897576306240', '40', '<O>', '40', '<O>'], ['1897576703296', 'billion', '<O>', 'billion', '<O>'], ['1897576296512', 'US', '<O>', 'US', '<O>'], ['1897576698624', 'airtanker', '<O>', 'airtanker', '<O>'], ['1897576701760', 'contract', '<O>', 'contract', '<O>'], [], ['1897571439936', 'March', '<O>', 'March', '<O>'], ['1897576699840', '01', '<O>', '01', '<O>'], ['1897571438912', '2008', '<O>', '2008', '<O>'], []]


In [52]:
ids = []
for line in lines:
    if len(line) ==5 and line[1] != line[3]:
        ids.append([line[0], line[1], line[3]])
    ids
    
print(ids)

[['1897576447104', '(', '(MRO)'], ['1897576447232', 'MRO', 'operations'], ['1897576447296', ')', 'across'], ['1897576447360', 'operations', 'the'], ['1897576447424', 'across', 'aircraft'], ['1897576447552', 'the', "'s"], ['1897576447616', 'aircraft', 'fifty-year'], ['1897576447808', "'s", 'service'], ['1897576447936', 'fifty-year', 'life'], ['1897576464448', 'service', 'could'], ['1897576480000', 'life', 'push'], ['1897576480320', 'could', 'the'], ['1897576480448', 'push', 'value'], ['1897576480384', 'the', 'of'], ['1897576694080', 'value', 'the'], ['1897576698944', 'of', 'contract'], ['1897576697728', 'the', 'as'], ['1897576697536', 'contract', 'high'], ['1897576695552', 'high', 'US$'], ['1897576707648', 'as', '100'], ['1897576708096', 'US$', 'billion'], ['1897576707968', '100', 'Indonesia'], ['1897576708032', 'billion', "'s"], ['1897576708160', 'Indonesia', 'transport'], ['1897576708224', "'s", 'minister'], ['1897576708288', 'transport', 'tells'], ['1897576708352', 'minister', 'airli

In [53]:
len(ids)

412

In [54]:
for item in ids:
    print(f'{item[0]}\t{item[1]}\t{item[2]}')

1897576447104	(	(MRO)
1897576447232	MRO	operations
1897576447296	)	across
1897576447360	operations	the
1897576447424	across	aircraft
1897576447552	the	's
1897576447616	aircraft	fifty-year
1897576447808	's	service
1897576447936	fifty-year	life
1897576464448	service	could
1897576480000	life	push
1897576480320	could	the
1897576480448	push	value
1897576480384	the	of
1897576694080	value	the
1897576698944	of	contract
1897576697728	the	as
1897576697536	contract	high
1897576695552	high	US$
1897576707648	as	100
1897576708096	US$	billion
1897576707968	100	Indonesia
1897576708032	billion	's
1897576708160	Indonesia	transport
1897576708224	's	minister
1897576708288	transport	tells
1897576708352	minister	airlines
1897576708416	tells	not
1897576708480	airlines	to
1897576708544	not	buy
1897576708608	to	European
1897576708672	buy	aircraft
1897576708736	European	due
1897576708800	aircraft	to
1897576708864	due	EU
1897576708928	to	ban
1897576708992	EU	<>.
1897576709056	ban	<>,
1897576796736	A380	A
1897576